In [4]:
import numpy as np
import pandas as pd
from function.function_select import select_index_batch
from function.function_select import select_index
def SameValueCount(x):
    y = np.unique(x)
    X = np.tile(x, (len(y), 1)).T
    Y = np.tile(y, (len(x), 1))
    k = np.arange(len(x))+1
    match_ = np.equal(X, Y)
    match_not = np.logical_not(match_)
    A = match_ * np.reshape(k, (-1,1))
    B = match_not * (len(x)*3)
    C = A+B
    min_C = np.min(C, axis=0)
    min_C = np.tile(min_C, (len(x), 1))
    D = C-min_C
    Final = (D[np.less_equal(D, len(x))])
    return Final
top_path = './../../'
NoIV_path = top_path + 'Data/Organized/NoIV/'

expirys = ['NearbyMonth', 'TwoMonth', 'ThreeMonth', 'SixMonth', 'NineMonth']
TXO_Data = pd.read_csv(NoIV_path + 'TXO_LTDR.csv',  encoding='Big5', index_col=False)
TXO_Data = TXO_Data[TXO_Data['結算價'] != 0] #篩掉到期天數為0的資料
TXO_matrix = np.array(TXO_Data)

dates = np.unique(TXO_matrix[:,:2].astype(str), axis=0)


counts = SameValueCount(dates[:,0])


k = np.array(['000000000000000']*len(TXO_Data))
for i in range(len(expirys)):
    expiry_dates = dates[np.equal(counts, i)]
    rows_dates, rows_TXO = select_index_batch(expiry_dates, TXO_matrix[:, :2].astype(str), \
                                            len(expiry_dates), 10000)
    k[rows_TXO] = expirys[i]
TXO_matrix_new = np.hstack((TXO_matrix, np.reshape(k, (-1,1))))
columns = TXO_Data.columns.to_numpy()
columns_new = np.hstack((columns, np.array(['到期月份種類'])))
TXO_Data_new = pd.DataFrame(data=TXO_matrix_new, columns=columns_new)
TXO_Data_new.to_csv(NoIV_path + 'TXO_Eclassify.csv', index=False, encoding='Big5')

SameValueCount的實際操作

In [ ]:
import numpy as np
x = np.array([1, 1, 1, 2, 2, 3])
y = np.unique(x)

X = np.tile(x, (len(y), 1)).T
'''
X = [[1 1 1]
     [1 1 1]
     [1 1 1]
     [2 2 2]
     [2 2 2]
     [3 3 3]]
''' 
#print(X)
Y = np.tile(y, (len(x), 1))
'''
Y = [[1 2 3]
     [1 2 3]
     [1 2 3]
     [1 2 3]
     [1 2 3]
     [1 2 3]]
'''
#print(Y)
k = np.arange(len(x))+1

#K = np.tile(k, (len(y), 1)).T
'''
K = [[1 1 1]
     [2 2 2]
     [3 3 3]
     [4 4 4]
     [5 5 5]
     [6 6 6]]
'''
#print(K)
match_ = np.equal(X, Y)
'''
match_ = [[ True False False]
          [ True False False]
          [ True False False]
          [False  True False]
          [False  True False]
          [False False  True]]
'''
#print(match_)
match_not = np.logical_not(match_)
#print(match_not)

#print(match_ * np.reshape(k, (-1,1)))

A = match_ * np.reshape(k, (-1,1))
#print(A)
'''
A =      [[1 0 0]
          [2 0 0]
          [3 0 0]
          [0 4 0]
          [0 5 0]
          [0 0 6]]
'''
B = match_not * (len(x)*3)

'''
B =      [[ 0 18 18]
          [ 0 18 18]
          [ 0 18 18]
          [18  0 18]
          [18  0 18]
          [18 18  0]]
'''
#print(B)
C = A+B
#print(C)

'''
C =      [[ 1 18 18]
          [ 2 18 18]
          [ 3 18 18]
          [18  4 18]
          [18  5 18]
          [18 18  6]]
'''
min_C = np.min(C, axis=0)
min_C = np.tile(min_C, (len(x), 1))

'''
min_C =   [[1 4 6]
           [1 4 6]
           [1 4 6]
           [1 4 6]
           [1 4 6]
           [1 4 6]]
'''
#print(min_C)
D = C-min_C
'''
D =   [[ 0 14 12]
       [ 1 14 12]
       [ 2 14 12]
       [17  0 12]
       [17  1 12]
       [17 14  0]]
'''
#print(D)
Final = (D[np.less_equal(D, len(x))])
'''
Final = [0 1 2 0 1 0]
'''

print(Final)
